# Aula 06 - NER

Professor: Luiz Frias 
- Email: l.frias@poli.ufrj.br
- Linkedin: in/luizfrias/
- Twitter: @lfdefrias

**Objetivo:** entendimento de como identificar entididades com o Spacy.

In [1]:
import spacy
from spacy import displacy

import pandas as pd

from newsplease import NewsPlease

In [2]:
nlp = spacy.load("pt_core_news_md")

Vamos usar uma notícia de exemplo para identificar as entidades 

In [129]:
article = NewsPlease.from_url("https://economia.uol.com.br/colunas/carla-araujo/2020/09/21/tst-julgamento-fim-greve-correios-manter-parte-direitos-reajuste.htm")

In [75]:
print (article.maintext)

Jornalista formada em 2003 pela FIAM, com pós-graduação na Fundação Cásper Líbero e MBA em finanças, começou a carreira repórter de agronegócio e colaborou com revistas segmentadas. Na Agência Estado/Broadcast foi repórter de tempo real por dez anos em São Paulo e também em Brasília, desde 2015. Foi pelo grupo Estado que cobriu o impeachment da presidente Dilma Rousseff. No Valor Econômico, acompanhou como setorista do Palácio do Planalto o fim do governo Michel Temer e a chegada de Jair Bolsonaro à Presidência.
O Tribunal Superior do Trabalho decidiu, em julgamento realizado nesta segunda-feira, encerrar a greve dos funcionários dos Correios, que acontece desde 17 de agosto. De acordo com a decisão, os empregados devem voltar ao trabalho amanhã, sob pena de multa de R$ 100 mil por dia.
A empresa comemorou a decisão. Já o sindicato rejeitou o acordo e recomendou que os funcionários mantenham a greve. Devem ser feitas assembleias hoje e amanhã para votação.
A decisão de hoje representou

### Rule-based matcher

O Spacy possibilita encontrar pedaços de texto com base em regras. São duas as classes: PhraseMatcher, que encontra pedaços exatos de texto; Matcher, que encontra trechos do documento através de padrões específicos. 

#### Phrase Matcher

Vamos utilizar o PhraseMatcher para encontrar estados e municípios no texto.

In [5]:
estados = pd.read_csv('in/estados.csv')
estados.head(1)

,codigo_uf,uf,nome,latitude,longitude
0,11,RO,Rondônia,-10.83,-63.34


In [6]:
municipios = pd.read_csv('in/municipios.csv')
municipios.head(1)

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,0,52


In [7]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

estados_uf_pattern = [nlp(estado) for estado in estados.uf.tolist()]
estados_name_pattern = [nlp(estado) for estado in estados.nome.tolist()]
municipios_name_pattern = [nlp(city) for city in municipios.nome.tolist()]

matcher.add("ESTADO_UF", estados_uf_pattern)
matcher.add("ESTADO_NOME", estados_name_pattern)
matcher.add("MUNICIPIO_NOME", municipios_name_pattern)

doc = nlp(article.maintext)

for match_id, start, end in matcher(doc):
    span = doc[start:end]
    print(nlp.vocab.strings[match_id], start, end, span.text)

ESTADO_NOME 44 46 São Paulo
MUNICIPIO_NOME 44 46 São Paulo
MUNICIPIO_NOME 49 50 Brasília
MUNICIPIO_NOME 77 78 Planalto
MUNICIPIO_NOME 288 289 Floriano
MUNICIPIO_NOME 288 290 Floriano Peixoto
MUNICIPIO_NOME 788 789 Floriano
MUNICIPIO_NOME 788 790 Floriano Peixoto
MUNICIPIO_NOME 838 839 Floriano
MUNICIPIO_NOME 886 887 Floriano


Podemos ver todos os estados/munípios encontrados. Como o PhraseMatcher não é um modelo probabilístico, o contexto em que a palavra aparece não influencia no resultado. Assim, Floriano, Floriano Peixoto e Planalto são encontradas, mesmo que no texto original elas não representem localidades.

In [17]:
pd.merge(
    estados,
    municipios[municipios.nome.str.contains('Planalto')],
    on='codigo_uf'
)

,codigo_uf,uf,nome_x,latitude_x,longitude_x,codigo_ibge,nome_y,latitude_y,longitude_y,capital
0,29,BA,Bahia,-13.29,-41.71,2925006,Planalto,-14.6654,-40.4718,0
1,35,SP,São Paulo,-22.19,-48.79,3539608,Planalto,-21.0342,-49.9330,0
2,41,PR,Paraná,-24.89,-51.55,4119806,Planalto,-25.7211,-53.7642,0
3,42,SC,Santa Catarina,-27.45,-50.95,4213153,Planalto Alegre,-27.0704,-52.8670,0
4,43,RS,Rio Grande do Sul,-30.17,-53.50,4314704,Planalto,-27.3297,-53.0575,0
5,43,RS,Rio Grande do Sul,-30.17,-53.50,4317756,Santo Antônio do Planalto,-28.4030,-52.6992,0
6,51,MT,Mato Grosso,-12.64,-55.42,5106455,Planalto da Serra,-14.6518,-54.7819,0
7,52,GO,Goiás,-15.98,-49.86,5215256,Novo Planalto,-13.2424,-49.5060,0


In [16]:
pd.merge(
    estados,
    municipios[municipios.nome.str.contains('Floriano')],
    on='codigo_uf'
)

,codigo_uf,uf,nome_x,latitude_x,longitude_x,codigo_ibge,nome_y,latitude_y,longitude_y,capital
0,22,PI,Piauí,-6.60,-42.28,2203909,Floriano,-6.77182,-43.0241,0
1,32,ES,Espírito Santo,-19.19,-40.34,3203346,Marechal Floriano,-20.41590,-40.6700,0
2,43,RS,Rio Grande do Sul,-30.17,-53.50,4308250,Floriano Peixoto,-27.86140,-52.0838,0


#### Matcher

O Matcher é uma classe que oferece funcionalidades mais genéricas. Abaixo podemos ver todas as ocorrências para cada um dos padrões montados.

In [8]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

matcher.add("Unigrama Proprio", [[{'POS': 'PROPN'}]])
matcher.add("Bigrama Proprio", [[{'POS': 'PROPN'}, {'POS': 'PROPN'}]])
matcher.add("Trigrama Proprio", [[{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]])
matcher.add("Trigrama Proprio - Variacao", [[{'POS': 'PROPN'}, {'POS': 'DET'}, {'POS': 'PROPN'}]])

doc = nlp(article.maintext)

for match_id, start, end in matcher(doc):
    span = doc[start:end]
    print(span.text)

FIAM
Fundação
Fundação Cásper
Cásper
Fundação Cásper Líbero
Cásper Líbero
Líbero
MBA
Agência
Agência Estado
Estado
Broadcast
São
São Paulo
Paulo
Brasília
Dilma
Dilma Rousseff
Rousseff
Valor
Valor Econômico
Econômico
Palácio
Michel
Michel Temer
Temer
Jair
Jair Bolsonaro
Bolsonaro
Tribunal
Tribunal Superior
Superior
Superior do Trabalho
Trabalho
Comunicações
Fabio
Fabio Faria
Faria
Floriano
Floriano Peixoto
Peixoto
Kátia
Kátia Arruda
Arruda
Ives
Ives Gandra
Gandra
Ives Gandra Filho
Gandra Filho
Filho
Jair
Jair Bolsonaro
Bolsonaro
Ives
Ives Gandra
Gandra
estatal
CLT
CLT
CLT
Kátia
Kátia Arruda
Arruda
Justiça
Justiça do Trabalho
Trabalho
Justiça
Justiça do Trabalho
Trabalho
estatal
Floriano
Floriano Peixoto
Peixoto
Floriano
Floriano
Brasil


Acima usamos apenas o uso atributo POS, ver todas possibildades de uso em: https://spacy.io/usage/rule-based-matching#adding-patterns-attributes

Um outro elemento do Matcher é o `op`, que determina o número de ocorrências para cada padrão. Dessa forma, todos os padrões acima, podemos ser resumidos em um só padrão, como o abaixo.

In [9]:
matcher = Matcher(nlp.vocab)

matcher.add("Entidade Matcher", [[
    {'POS': 'PROPN', 'op': '+'}, 
    {'POS': 'DET', 'op': '?'}, 
    {'POS': 'PROPN', 'op': '?'},
    {'POS': 'DET', 'op': '?'}, 
    {'POS': 'PROPN', 'op': '?'},
    {'POS': 'DET', 'op': '?'}, 
    {'POS': 'PROPN', 'op': '+'}
]])

doc = nlp(article.maintext)

for match_id, start, end in matcher(doc):
    span = doc[start:end]
    print(start, end, span.text)

10 12 Fundação Cásper
10 13 Fundação Cásper Líbero
11 13 Cásper Líbero
31 33 Agência Estado
44 46 São Paulo
64 66 Dilma Rousseff
68 70 Valor Econômico
82 84 Michel Temer
88 90 Jair Bolsonaro
95 97 Tribunal Superior
96 99 Superior do Trabalho
95 99 Tribunal Superior do Trabalho
278 280 Fabio Faria
288 290 Floriano Peixoto
347 349 Kátia Arruda
371 373 Ives Gandra
371 374 Ives Gandra Filho
372 374 Gandra Filho
382 384 Jair Bolsonaro
464 466 Ives Gandra
665 667 Kátia Arruda
679 682 Justiça do Trabalho
756 759 Justiça do Trabalho
788 790 Floriano Peixoto


Outra possibilidade de atributo é o SHAPE, que pode ser utilizado para recuperar os números de 4 dígitos, por exemplo.

In [10]:
matcher = Matcher(nlp.vocab)

matcher.add("Ano Matcher", [[
    {'SHAPE': 'dddd'}, 
]])

doc = nlp(article.maintext)

for match_id, start, end in matcher(doc):
    span = doc[start:end]
    print(span.text)

2003
2015


O Matcher espera uma função de callback, que serve para modificar os atributos do documento. Um possível uso para ela é modificar as entidades encontradas no documento com novos padrões, como no exemplo abaixo.

In [11]:
from spacy.tokens import Span

def on_match(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = Span(doc, start, end, label="ANO")
    doc.ents += (entity,)

matcher = Matcher(nlp.vocab)

matcher.add("Ano Matcher", on_match, [
    {'SHAPE': 'dddd'}, 
])

doc = nlp(article.maintext)

for match_id, start, end in matcher(doc):
    span = doc[start:end]

print (doc.ents)

(2003, FIAM, Fundação Cásper Líbero, MBA, Agência Estado, Broadcast, São Paulo, Brasília, 2015, Estado, Dilma Rousseff, Valor Econômico, Palácio do Planalto, Michel Temer, Jair Bolsonaro, Presidência, Tribunal Superior do Trabalho, Correios, Corte, Comunicações, Fabio Faria, Correios, Floriano Peixoto, Kátia Arruda, Ives Gandra Filho, Jair Bolsonaro, Corte, Correios, Ives Gandra, CLT, CLT, Correios, CLT, Kátia Arruda, Correios, Justiça do Trabalho, Justiça do Trabalho, Correios, Floriano Peixoto, Floriano, Floriano, Brasil, Correios)


### Named Entity Recognition (NER)

Apesar das classes acima identificarem certos padrões no texto, o contexto em que eles aparecem não é levado em consideração. Por esse motivo, vemos algumas falhas, como a identificação de palavras que não deveriam ter sido identificadas em primeiro lugar (como os nomes de munícipio no exemplo acima neste caderno).

O Spacy conta com um modelo de identificação de entidades que pode funcionar melhor em muitos casos. A seguir vamos explorar seu funcionamento.

Por padrão, o pipeline de processo feito pelo Spacy a partir do uso do objeto `nlp` faz a identificação de entidades, divididas em LOC, ORG, PER e MISC. Abaixo ainda utilizamos o nosso `matcher` para modificar as entidades do documento e adicionar todos os números de 4 dígitos à entidade ANO.

In [12]:
doc = nlp(article.maintext)
matcher(doc);

In [13]:
displacy.render(doc, style="ent", jupyter=True, options={'distance': 100})

### EntityRuler

Uma opção melhor do que usar o callback do Matcher para modificar as entidades de um documento é utilizar a classe EntityRuler. Além de possibilitar encontrar trechos e padrões, ainda resolve um problema comum: entidades que se sobrepõem. Considere o exemplo de `PhraseMatcher` deste caderno, onde Floriano e Floriano Peixoto foram encontradas. Como ambas as palavras representam municípios brasileiros, o Matcher retornou as duas. Mas Floriano Peixoto contém Floriano e portanto elas não podem representar duas entidades. 

A `EntityRuler` cuida da tarefa de manter apenas a entidades de maior comprimento, além de automaticamente modificar o atributo `ents` do documento.

In [25]:
nlp_pt = spacy.load("pt_core_news_md")

In [26]:
nlp_pt.pipe_names

['tagger', 'parser', 'ner']

In [82]:
nlp_pt.remove_pipe('entity_ruler')

('entity_ruler', <spacy.pipeline.entityruler.EntityRuler at 0x13c0cde48>)

In [84]:
from spacy.pipeline import EntityRuler

ruler = EntityRuler(nlp_pt, overwrite_ents=True, validate=True)
patterns = [
    {"label": "ANO", "pattern": [{"SHAPE": "dddd"},]},
    {"label": "DIN", "pattern": [{"ORTH": "R$"}, {"LIKE_NUM": True}]},
    {"label": "DIN", "pattern": [{"ORTH": "R$"}, {"LIKE_NUM": True}, {"LIKE_NUM": True}]}
]
ruler.add_patterns(patterns)
nlp_pt.add_pipe(ruler)

In [85]:
nlp_pt.pipe_names

['tagger', 'parser', 'ner', 'entity_ruler']

In [86]:
doc = nlp_pt(article.maintext)

displacy.render(doc, style="ent", jupyter=True, options={'distance': 100})

### Treinando o NER

Como podemos observar acima, o NER do Spacy não foi capaz de classificar "ministro das Comunicações" como PER. Vamos  demonstrar como isso poderia ser corrigido.

Precisamos de, no mínimo, algumas centenas de observações para um treinamento do NER. Como não temos os recursos para fazer isso durante uma aula, vamos nos limitar a um número bem menor de anotações, o que provavelmente vai impactar na acurácia do modelo.

In [87]:
ner = nlp.get_pipe("ner")

In [88]:
import requests
url = ('http://newsapi.org/v2/everything?q=ministro&sources=google-news-br,globo&apiKey=%s' % open('google-api-key.txt').read())
response = requests.get(url)
print (response.json())

{'status': 'ok', 'totalResults': 1256, 'articles': [{'source': {'id': 'globo', 'name': 'Globo'}, 'author': None, 'title': 'Deputado pede que ministro explique afirmações sobre privatização dos Correios', 'description': 'O deputado federal André Figueiredo (PDT/CE) entrou com um requerimento de informações da Câmara (RIC nº 1218/2020) acerca das declarações feitas pelo Ministro das Comunicações, Fábio Faria, tratando da intenção de desestatização dos Correios. Na noite de qua…', 'url': 'https://valor.globo.com/brasil/noticia/2020/09/24/deputado-pede-que-ministro-explique-afirmacoes-sobre-privatizacao-dos-correios.ghtml', 'urlToImage': 'https://s2.glbimg.com/KshyJ3l9So6j0WObBYj2h0rORNk=/1200x/smart/filters:cover():strip_icc()/i.s3.glbimg.com/v1/AUTH_63b422c2caee4269b8b34177e8876b93/internal_photos/bs/2019/1/q/hsPfwXQlyv3MEEqwzfXw/1111707-mac-edit-12031807463.jpg', 'publishedAt': '2020-09-24T21:31:03Z', 'content': 'O deputado federal André Figueiredo (PDT/CE) entrou com um requerimento de

In [89]:
for article in response.json()['articles']:
    print (article['url'])
    break

https://valor.globo.com/brasil/noticia/2020/09/24/deputado-pede-que-ministro-explique-afirmacoes-sobre-privatizacao-dos-correios.ghtml


In [90]:
%%time

articles = []
counter = 0
for article in response.json()['articles']:
    print ('Fetching "%s"...' % article['title'])
    news_obj = NewsPlease.from_url(article['url'])
    articles.append(news_obj.maintext)
    counter += 1
    if counter > 20:
        break

Fetching "Deputado pede que ministro explique afirmações sobre privatização dos Correios"...
Fetching "Petrobras tem 2° voto no STF contra venda de refinarias, do ministro Lewandowski"...
Fetching "Fundo para garantir perdas em reforma tributária seria imprudente, União não é saco sem fundo, diz Guedes"...
Fetching "Suga, do Japão, promete priorizar crescimento antes de reforma fiscal"...
Fetching "Modi promete usar produção da vacina na Índia para ajudar 'toda a humanidade'"...
Fetching "Fundo para garantir perdas em reforma tributária pode quebrar o país, alerta Guedes"...
Fetching "Guedes elogia Bolsonaro e atribui fim do Renda Brasil à distorção de informação da mídia"...
Fetching "Pazuello nomeia veterinário para dirigir Programa Nacional de Imunizações"...
Fetching "UE acelera planejamento para saída sem acordo enquanto Reino Unido se recusa a ceder em impasse"...
Fetching "Pazuello nomeia veterinário para dirigir Departamento de Imunizações"...
Fetching "Reforma administrativa s

In [94]:
sents = []
for art in articles:
    for sent in nlp(art).sents:
        if 'ministro da' in sent.lower_ or 'ministro do' in sent.lower_:
            print (sent)
            sents.append(sent)

O deputado federal André Figueiredo (PDT/CE) entrou com um requerimento de informações da Câmara (RIC nº 1218/2020) acerca das declarações feitas pelo Ministro das Comunicações, Fábio Faria, tratando da intenção de desestatização dos Correios.
Por Marcela Ayres
BRASÍLIA (Reuters) - O ministro da Economia, Paulo Guedes, classificou como imprudente a ideia de se instituir um fundo com recursos federais para garantir perdas de Estados e municípios com a reforma tributária, alegando que a União não é um "saco sem fundo" e que é necessário ter juízo em um momento de fragilidade das contas públicas.

Por Leika Kihara e Chris Gallagher
TÓQUIO (Reuters) - Yoshihide Suga, a caminho de se tornar o próximo primeiro-ministro do Japão, disse que manterá a política do atual premiê, Shinzo Abe, priorizando o crescimento econômico antes dos esforços para consertar as finanças esfarrapadas do país.

Por Marcela Ayres
BRASÍLIA (Reuters) - O ministro da Economia, Paulo Guedes, classificou como imprudente

In [99]:
matcher = Matcher(nlp.vocab)

matcher.add("Ministro Matcher", [[
    {'LOWER': 'ministro'},
    {'POS': 'DET'},
    {}
], [
    {'LOWER': 'ministra'},
    {'POS': 'DET'},
    {}
]])

In [115]:
for sent in sents:
    for match_id, start, end in matcher(sent):
        span = sent[start:end]
        print ('("%s", {"entities": [(%d, %d, "PER")]}),' % (sent.text.replace('"', ''), span.start_char, span.end_char))

("O deputado federal André Figueiredo (PDT/CE) entrou com um requerimento de informações da Câmara (RIC nº 1218/2020) acerca das declarações feitas pelo Ministro das Comunicações, Fábio Faria, tratando da intenção de desestatização dos Correios.", {"entities": [(151, 176, "PER")]}),
("Por Marcela Ayres
BRASÍLIA (Reuters) - O ministro da Economia, Paulo Guedes, classificou como imprudente a ideia de se instituir um fundo com recursos federais para garantir perdas de Estados e municípios com a reforma tributária, alegando que a União não é um saco sem fundo e que é necessário ter juízo em um momento de fragilidade das contas públicas.
", {"entities": [(66, 86, "PER")]}),
("Por Marcela Ayres
BRASÍLIA (Reuters) - O ministro da Economia, Paulo Guedes, classificou como imprudente a ideia de se instituir um fundo com recursos federais para garantir perdas de Estados e municípios com a reforma tributária, alegando que a União não é um saco sem fundo e que é necessário ter juízo em um momento d

In [117]:
TRAIN_DATA = [
    ("O deputado federal André Figueiredo (PDT/CE) entrou com um requerimento de informações da Câmara (RIC nº 1218/2020) acerca das declarações feitas pelo Ministro das Comunicações, Fábio Faria, tratando da intenção de desestatização dos Correios.", {"entities": [(151, 176, "PER")]}),
    ("BRASÍLIA (Reuters) - O ministro da Economia, Paulo Guedes, classificou como imprudente a ideia de se instituir um fundo com recursos federais para garantir perdas de Estados e municípios com a reforma tributária, alegando que a União não é um saco sem fundo e que é necessário ter juízo em um momento de fragilidade das contas públicas.", {"entities": [(66, 86, "PER")]}),
    ("BRASÍLIA (Reuters) - O ministro da Economia, Paulo Guedes, atribuiu nesta terça-feira a decisão do presidente Jair Bolsonaro de acabar com o Renda Brasil a uma posição política que considerou correta após distorção de informação feita pela mídia.", {"entities": [(66, 86, "PER")]}),
    ("Nesta terça-feira, o ministro da Economia também defendeu que a instituição de um imposto sobre transações digitais poderia, ao abraçar uma nova base ampla, permitir a desoneração de outros tributos, em especial o imposto sobre a folha de pagamento, que considerou muito mais cumulativo e destrutivo.", {"entities": [(5566, 5586, "PER")]}),
    ("O presidente Jair Bolsonaro deu aval e a reforma administrativa será enviada à Câmara dos Deputados na quinta-feira, afirmou o ministro da Economia, Paulo Guedes, em audiência pública do Congresso realizada virtualmente nesta terça-feira.", {"entities": [(173, 193, "PER")]}),
    ("BRASÍLIA (Reuters) - O ministro da Economia, Paulo Guedes, disse nesta quarta-feira que a equipe econômica está submetendo à apreciação dos líderes no Congresso contribuições para a reforma tributária.", {"entities": [(68, 88, "PER")]}),
    ("A China deverá registrar nas próximas semanas uma colheita abundante de grãos, impulsionada pelas boas condições de cultivo e por um aumento nas áreas de plantio, disse o ministro da Agricultura e Assuntos Rurais do país, Han Changfu, em um comunicado nesta quarta-feira.", {"entities": [(215, 238, "PER")]}),
    ("O ministro da Economia, Paulo Guedes, afirmou nesta segunda-feira que o Brasil é um país que precisa criar emprego em massa e, quanto à solução para bancar a desoneração da folha de pagamento para incentivar a investida, sinalizou que a decisão ainda não foi tomada.", {"entities": [(48, 68, "PER")]}),
    ("BRASÍLIA - Após o presidente Jair Bolsonaro vetar a ideia da equipe econômica de congelar aposentadorias e pensões para abrir espaço no Orçamento para o Renda Brasil, o ministro da Economia, Paulo Guedes, disse que a ideia foi mal interpretada pela imprensa e afirmou que o cartão vermelho mencionado por Bolsonaro não foi para ele.", {"entities": [(202, 222, "PER")]}),
    ("O ministro da Economia, Paulo Guedes, afirmou nesta terça-feira que a queda do PIB registrada no segundo trimestre do ano é um som distante que reflete os efeitos da pandemia da Covid-19 e frisou que o Brasil já deu início a uma recuperação em V.", {"entities": [(48, 68, "PER")]}),
]

In [123]:
def train_spacy(_nlp, data, iterations):
    TRAIN_DATA = data
    
    if 'ner' not in _nlp.pipe_names:
        ner = _nlp.create_pipe('ner')
        _nlp.add_pipe(ner, last=True)
    else:
        ner = _nlp.get_pipe('ner')
       
    # Adiciona os labels ao NER
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # Desabilita tudo o que nao e NER durante o treinamento
    other_pipes = [pipe for pipe in _nlp.pipe_names if pipe != 'ner']
    with _nlp.disable_pipes(*other_pipes):  
        optimizer = _nlp.begin_training()
        for itn in range(iterations):
            print("Iteração " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                _nlp.update(
                    [text],  
                    [annotations],  
                    drop=0.2,  
                    sgd=optimizer,  
                    losses=losses)
            print(losses)
    return _nlp

In [126]:
nlp_new = train_spacy(spacy.blank('pt'), TRAIN_DATA, 20)

Iteração 0


/opt/miniconda3/envs/text/lib/python3.6/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "O presidente Jair Bolsonaro deu aval e a reforma a..." with entities "[(173, 193, 'PER')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/opt/miniconda3/envs/text/lib/python3.6/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "O ministro da Economia, Paulo Guedes, afirmou nest..." with entities "[(48, 68, 'PER')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/opt/miniconda3/envs/text/lib/python3.6/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "BRASÍLIA - 

{'ner': 193.65341413026908}
Iteração 1
{'ner': 2.05995669171867}
Iteração 2
{'ner': 2.0000000000006417}
Iteração 3
{'ner': 1.9999095201789625}
Iteração 4
{'ner': 2.372197449527375}
Iteração 5
{'ner': 0.8878550938477398}
Iteração 6
{'ner': 0.8761641066129247}
Iteração 7
{'ner': 0.5827018086397743}
Iteração 8
{'ner': 22.587446309931718}
Iteração 9
{'ner': 3.6422087861566275}
Iteração 10
{'ner': 2.7553926060805773}
Iteração 11
{'ner': 2.487336533364023}
Iteração 12
{'ner': 1.3143227101607757}
Iteração 13
{'ner': 2.0034914584601466}
Iteração 14
{'ner': 4.404966099223653}
Iteração 15
{'ner': 2.662613058853181}
Iteração 16
{'ner': 1.7601307713988457}
Iteração 17
{'ner': 4.4746766272432135}
Iteração 18
{'ner': 1.0787051684450357}
Iteração 19
{'ner': 2.0429058453294906}


In [129]:
article = NewsPlease.from_url("https://economia.uol.com.br/colunas/carla-araujo/2020/09/21/tst-julgamento-fim-greve-correios-manter-parte-direitos-reajuste.htm")

In [132]:
doc = nlp_new(article.maintext)

displacy.render(doc, style="ent", jupyter=True, options={'distance': 100})

In [54]:
nlp.to_disk('out/custom_nlp.pkl')